In [1]:
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from ultralytics.data.converter import coco80_to_coco91_class
from ultralytics.utils.ops import xyxy2ltwh
from tqdm import tqdm
from ultralytics import YOLO
import json
from yolo_triton_client import TritonYOLOInference
from PIL import Image
import numpy as np

model = YOLO("yolo11x.pt")
_ = model.predict(Image.fromarray(np.zeros((224,224))))


0: 640x640 (no detections), 23.9ms
Speed: 9.3ms preprocess, 23.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


# 1. Evaluating with ultralytics torch YOLOv11x model

In [2]:
_ = model.val(data="coco.yaml", augment=False)

Ultralytics 8.3.228 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

WARNING ⚠️ Dataset 'coco.yaml' images not found, missing path '/mnt/d/Sergey/ML/datasets/coco/val2017.txt'
Unzipping /mnt/d/Sergey/ML/datasets/coco2017labels-segments.zip to /mnt/d/Sergey/ML/datasets/coco...: 100% ━━━━━━━━━━━━ 122232/122232 173.7files/s 11:44<0.0s
Unzipping /mnt/d/Sergey/ML/datasets/coco/images/val2017.zip to /mnt/d/Sergey/ML/datasets/coco/images/val2017...: 100% ━━━━━━━━━━━━ 5001/5001 127.3files/s 39.3s0.1s:605
Unzipping /mnt/d/Sergey/ML/datasets/coco/images/test2017.zip to /mnt/d/Sergey/ML/datasets/coco/images/test2017...: 100% ━━━━━━━━━━━━ 40671/40671 109.3files/s 6:12<0.0s5
Unzipping /mnt/d/Sergey/ML/datasets/coco/images/train2017.zip to /mnt/d/Sergey/ML/datasets/coco/images/train2017...: 100% ━━━━━━━━━━━━ 118288/118288 136.1files/s 14:29<0.1s
Dataset download success ✅ (2738.5s), saved to /mnt/d/Sergey/ML/datasets

val: Fast image access ✅ (ping: 4.6±1.9 ms, read: 19.

# 2. Evaluating with triton endpoint at localhost:8000/yolo

## Collecting images with annotations (48 out of 5000 images in val2017 are not annotated)

In [3]:
with open('../../datasets/coco/annotations/instances_val2017.json', 'r') as f:
    data = json.load(f)

i2gt = {}
for item in data['annotations']:
    if item['image_id'] not in i2gt:
        i2gt[item['image_id']] = []
    item = {x: item[x] for x in {'image_id', 'bbox', 'category_id'}}
    i2gt[item['image_id']].append(item)


## Running evaluation with triton

In [4]:
with TritonYOLOInference(
        url="localhost:8000",
        model_name="yolo",
        preprocess_fn=model.predictor.preprocess,
        conf_threshold=0.001,
        iou_threshold=0.7,
        max_det=300
) as inference_client:

    my_predictions = []

    for image_id in tqdm(i2gt):
        # Inference from image path
        predictions = inference_client.predict(
            f'../../datasets/coco/images/val2017/{image_id:012d}.jpg'
        )

        # Alternative: Inference from PIL Image
        # pil_img = Image.open(f'../../datasets/coco/images/val2017/{image_id:012d}.jpg')
        # predictions = inference_client.predict(pil_img)

        predictions = predictions.cpu().numpy()
        boxes = xyxy2ltwh(predictions[:, :4])
        cat_ids = [coco80_to_coco91_class()[int(x)] for x in predictions[:, -1]]
        scores = predictions[:, -2]

        for box, cat, score in zip(boxes, cat_ids, scores):
            my_predictions.append({
                'image_id': image_id,
                'file_name': f'{image_id:012d}.jpg',
                'category_id': cat,
                'bbox': box.tolist(),
                'score': score.item()
            })

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4952/4952 [04:15<00:00, 19.38it/s]


## Creating prediction file and computing metrics

In [5]:
with open('my_predictions.json', 'w') as f:
    json.dump(my_predictions, f)
    
annFile = '../../datasets/coco/annotations/instances_val2017.json'
coco_gt = COCO(annFile)
coco_dt = coco_gt.loadRes('my_predictions.json')

coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.52s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.24s).
Accumulating evaluation results...
DONE (t=4.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.536
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.587
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.691
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.388
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.641
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

# Result AP @[IoU=0.50:0.95]: 0.546 (torch) vs 0.536 (TensorRT+Triton)